In [1]:
import math
import random

In [7]:
class Value:
  def __init__(self, inData, inOperation = "", inChildren = None, inLabel = ""):
    self.data = float(inData)
    self.grad = 0.0
    self.operation = inOperation
    self.nBackward = lambda: None
    self.label = inLabel

    
    self.children = []
    if (inChildren is not None):
      self.children = [children for children in inChildren]
    
    
  def __repr__(self):
    if (self.label != ""):
      return f"Value[{self.label}]({self.data}), {self.operation if self.operation != '' else 'none'}, ( {len(self.children)} ), grad = {self.grad}"
    else:
      return f"{self.data}"

  # Operators

  def __add__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, "add", [self, other])

    def nBackward():
      self.grad += 1 * out.grad
      other.grad += 1 * out.grad

    out.nBackward = nBackward  
    return out

  def __radd__(self, other):
    return self + other
  
  def __neg__(self):
    out = Value(- self.data, "neg", [self])

    def nBackward():
      self.grad += -1 * out.grad

    out.nBackward = nBackward
    return out

  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, "mul", [self, other])
    
    def nBackward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad

    out.nBackward = nBackward
    return out

  def __rmul__(self, other):
    return self * other

  def __pow__(self, inPow):
    out = Value(self.data ** inPow, "pow", [self])

    def nBackward():
      self.grad += inPow * (self.data ** (inPow - 1)) * out.grad

    out.nBackward = nBackward
    
    return out

  def __truediv__ (self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data / other.data, "div", [self, other])

    def nBackward():
      self.grad += 1/other.data * out.grad
      other.grad += self.data * -1 * (other.data**-2) * out.grad

    out.nBackward = nBackward
    return out

  def __rtruediv__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    return other / self

  def tanh(self):
    out = Value(math.tanh(self.data), "tanh", [self])

    def nBackward():
      self.grad += (1 - math.tanh(self.data)**2) * out.grad
    
    out.nBackward = nBackward
    return out
    
  def backward(self):
    graph = []
    visited = set()
  
    def buildPath(inNode):

      if (inNode not in visited):
        visited.add(inNode)
        for child in inNode.children:
          buildPath(child)
          
        graph.append(inNode)

    buildPath(self)
    self.grad = 1.0

    for it in reversed(graph):
      it.nBackward()

  def zeroGrad(self):
    visited = set()
    
    def clear(inNode):
      if inNode not in visited:
        visited.add(inNode)
        inNode.grad = 0.0
        
        for child in inNode.children:
          clear(child)

    clear(self)

In [8]:
class Neuron:
  def __init__(self, inNNumber):
    self.bias = Value(random.uniform(-1.0, 1.0))
    self.weights = [Value(random.uniform(-1.0, 1.0)) for inN in range(inNNumber)]

  def __call__(self, inX):
    newSum = sum(wi * xi for wi, xi in zip(self.weights, inX)) + self.bias
    
    out = newSum.tanh()
    return out

  def parameters(self):
    return self.weights + [self.bias]
    
class Layer:
  def __init__(self, inNeurons, outNeurons):
    self.neurons = [Neuron(inNeurons) for neuron in range (outNeurons)]
    
  def __call__(self, inX):
    out = [neuron(inX) for neuron in self.neurons]
    return out

  def parameters(self):
    newList = []

    for neuron in self.neurons:
      newList = newList + neuron.parameters()
    return newList
    
class MLP:
  def __init__(self, layerInfo):
    self.network = []

    for layer in layerInfo:
      self.network.append(Layer(layer["in"], layer["out"]))

  def __call__(self, inX):
    x = inX
    for layer in self.network:
      x = layer(x)

    return x

  def parameters(self):
    newList = []

    for layer in self.network:
      newList = newList + layer.parameters()
    return newList

In [9]:
def printValues(inValue):
  
  valuesSet = set()

  def getValues(inValue):
    valuesSet.add(inValue)
    for value in inValue.children:
      getValues(value)

  getValues(inValue)
  for value in valuesSet:
    print(f"{value.data} - [{value.label}] - GRAD: {value.grad}")

In [10]:
# Grad test

a = Value(3.0, inLabel = "a")
b = Value(2.0, inLabel = "b")

c = a + b # 5
c.label = "c"

c.backward()

print(c)
printValues(c)

Value[c](5.0), add, ( 2 ), grad = 1.0
5.0 - [c] - GRAD: 1.0
3.0 - [a] - GRAD: 1.0
2.0 - [b] - GRAD: 1.0


In [11]:
# Neuron test
neuronTest = Neuron(1)
print("Neuron:\n", neuronTest([3]))

# Layer test
layerTest = Layer(3, 2)
print("Layer:\n", layerTest([3, 1, 2]))

# Network test
networkTest = MLP([
                    {"in": 3, "out": 5},
                    {"in": 5, "out": 5},
                    {"in": 5, "out": 5},
                    {"in": 5, "out": 5},
                    {"in": 5, "out": 3}
                  ])
print("Network:\n",networkTest([3.0, 2.0, 1.0]))

Neuron:
 0.8710554769395655
Layer:
 [0.7794741090294172, 0.8512819943703006]
Network:
 [0.9055854111291908, -0.23445857039349724, 0.9839809931103312]


In [20]:
# Learning test

yObj = [0.1, 0.3, 0.9]
y0 = [-0.8, 0.0, 0.1]
lr = 0.01

nEphocs = 100

nn = MLP([
          {"in": 3, "out": 5},
          {"in": 5, "out": 5},
          {"in": 5, "out": 5},
          {"in": 5, "out": 5},
          {"in": 5, "out": 3}
        ])

print(f"y0: {y0}")

for epoch in range (nEphocs):
  yModel = nn(y0)
  loss = sum((yPred + -yObj)**2 for yPred, yObj in zip(yModel, yObj))

  # Reset gradient
  loss.zeroGrad()
  loss.backward()

  # Step
  for parameter in nn.parameters():
    parameter.data += -lr * parameter.grad

  print(f"Epoca {epoch + 1}: {yModel}")

y0: [-0.8, 0.0, 0.1]
Epoca 1: [-0.7897126027060408, -0.3595113844988332, 0.8646505597760777]
Epoca 2: [-0.7715040803697628, -0.3022858532853632, 0.8554087754036226]
Epoca 3: [-0.750609558395929, -0.24509592640353614, 0.8451261783243041]
Epoca 4: [-0.7265972769647703, -0.18962517951866564, 0.8338460408156458]
Epoca 5: [-0.6990016738349308, -0.137380901561259, 0.8216528926682625]
Epoca 6: [-0.6673492445516942, -0.08950523529345579, 0.8086647013143883]
Epoca 7: [-0.6312111882557843, -0.04669010464100939, 0.7950283926646484]
Epoca 8: [-0.5902942512751227, -0.009193537187381844, 0.7809245339732136]
Epoca 9: [-0.5445737517502032, 0.023081765003153582, 0.766582121864012]
Epoca 10: [-0.4944522605016108, 0.05049119189289099, 0.7522975633924979]
Epoca 11: [-0.44089301940087344, 0.0735586984281811, 0.7384452408425608]
Epoca 12: [-0.38544496168323233, 0.09290138847886426, 0.7254638730424765]
Epoca 13: [-0.3300844504814366, 0.10916422325673862, 0.7138085889983331]
Epoca 14: [-0.27687835928385474, 0